In [82]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import re

In [2]:
def Clicking_on_element(driver,Xpath):
    dropdown_option = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH,Xpath)))
    return (dropdown_option.click())

In [3]:
def rep(text):
    return text.replace('\n',' ')

In [4]:
def get_data(url):
    try:
        option = webdriver.EdgeOptions()
        service = Service(executable_path='msedgedriver.exe')

        driver = webdriver.Edge(service=service)
        driver.get(url)

        time.sleep(2)

        Clicking_on_element(driver,'//*[@id="pop"]/div/section/button')

        phone_char = {}

        Phone_name = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[1]/section/div/div[2]/div[1]/div/h1')

        for n in Phone_name:
            name = n.text

        phone_char['Phone Name'] = name

        Phone_price = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[1]/section/div/div[2]/div[2]/div[2]/div/span')
        for pr in Phone_price:
            price = pr.text
        phone_char['Price'] = price

        discount = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[1]/section/div/div[2]/div[2]/div[2]/div/div/span[2]')

        for d in discount:
            dis = d.text

        phone_char['Discount percentage'] = dis

        original_price = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[1]/section/div/div[2]/div[2]/div[2]/div/div/span[1]')

        for o in original_price:
            orig = o.text
        phone_char['Price before discount'] = orig


        image = driver.find_elements(By.XPATH,'//*[@id="imgs"]/a/img')
        img = ''
        for i in image:
            img = i.get_attribute('src')
        phone_char['Image'] = img

        spec = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[2]/div[2]/div[1]/div[2]')
        coldf = []
        valuedf = []
        for s in spec:
            for col in range(1,12):
                coldf.append(s.find_element(By.XPATH,f'./p[{col}]').text)
            for value in range(1,12):
                valuedf.append(s.find_element(By.XPATH,f'./ul[{value}]').text)
        
        for i in range(len(valuedf)):
            valuedf[i] = rep(valuedf[i])

        for i in range(len(coldf)):
            phone_char[coldf[i]] = valuedf[i]

        key_spec = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[2]/div[2]/section[1]/div[2]/article[1]/div/div')
        key = ''
        for k in key_spec:
            key = k.text
        phone_char['Specifications'] = key

        no_reviews = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[2]/div[2]/section[2]/div[2]/div[2]/h2')
        num_rev = ' '
        
        for r in no_reviews:
            num_rev = r.text[17:-1]
        
        reviews = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[2]/div[2]/section[2]/div[2]/div[2]')
        num_rev = int(num_rev)
        rev_lst = []
        for r in reviews:
            for i in range(1,num_rev):
                rev = r.find_elements(By.XPATH,f'./article[{i}]/p')
                for j in rev:
                    rev_lst.append(j.text)
        rev_lst = '- '.join(rev_lst)

        phone_char['Reviews'] = rev_lst

        door_delivery = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[1]/div[2]/section/div[2]/article[1]/section/div/article[1]/div/div[2]')
        door = ''
        for d in door_delivery:
            door = d.text
        door = rep(door)
        phone_char['Door delivery'] = door

        pickup_station = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[1]/div[2]/section/div[2]/article[1]/section/div/article[2]/div/div[2]')
        pick = ''
        for p in pickup_station:
            pick = p.text
        pick = rep(pick)
        phone_char['Pickup_Station'] = pick


        return_policy = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[1]/div[2]/section/div[2]/article[2]/div/p')
        policy = ''
        for r in return_policy:
            policy = r.text
        policy = rep(policy)
        phone_char['Return_policy'] = policy

        warranty = driver.find_elements(By.XPATH,'//*[@id="jm"]/main/div[1]/div[2]/section/div[2]/article[3]/div/div')
        warr = ''
        for w in warranty:
            warr = w.text
        warr = rep(warr)
        phone_char['Warranty'] = warr
    except:
        pass
    driver.quit()
    return phone_char



In [5]:
def get_links(num):
    option = webdriver.EdgeOptions()
    service = Service(executable_path='msedgedriver.exe')
    driver = webdriver.Edge(service=service)
    url = 'https://www.jumia.com.eg/catalog/?q=mobile'
    driver.get(url)
    time.sleep(2)
    Clicking_on_element(driver,'//*[@id="pop"]/div/section/button')
    page = driver.find_elements(By.XPATH,f'//*[@id="jm"]/main/div[2]/div[3]/section/div[{num}]')
    links = []
    for link in page:
        for i in range(1,20):
            l = link.find_elements(By.XPATH,f'./article[{i}]/a')
            for j in l:
                links.append(j.get_attribute('href'))
    driver.quit()
    return links


In [9]:
links = get_links(1)
df = pd.DataFrame()
for i in links:
    new_df = pd.DataFrame(get_data(i),index=[0])
    df = pd.concat([df,new_df],ignore_index=True)

In [10]:
pd.set_option('display.max_columns',None)
df

,Phone Name,Price,Discount percentage,Price before discount,Image,Network,Body,Display,Platform,Memory,Main Camera,Selfie Camera,Sound,Communications,Features,Battery,Specifications,Reviews,Door delivery,Pickup_Station,Return_policy,Warranty,,
0,Samsung Galaxy A23 - 6.6-inch 128GB/6GB Dual S...,"EGP 7,636.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Samsung Galaxy A14 - 6.6 Inches - 4GB Ram – 12...,"EGP 5,399.00",4%,"EGP 5,600.00",https://eg.jumia.is/unsafe/fit-in/500x500/filt...,Technology: GSM / HSPA / LTE 2G bands: GSM 850...,Dimensions: 167.7 x 78 x 9.1 mm (6.60 x 3.07 x...,"Type: PLS LCD Size: 6.6 inches, 104.9 cm2 (~80...","OS: Android 13, One UI 5 Chipset: Mediatek MT6...",Card slot: microSDXC (dedicated slot) Internal...,"Quad: 50 MP, f/1.8, (wide), PDAF 5 MP, f/2.2, ...","Single: 13 MP, f/2.0, (wide) Video: 1080p@30fps",Loudspeaker: Yes 3.5mm jack: Yes,"WLAN: Wi-Fi 802.11 a/b/g/n/ac, dual-band, Wi-F...","Sensors: Fingerprint (side-mounted), accelerom...","Li-Po 5000 mAh, non-removable Charging: 15W wired","6.6 inches PLS LCD\n128GB Storage, 4 GB RAM\n5...",مناسب لاستخدامى- تليفون مميز في مجال الموبايلات,Free delivery Delivery by 18 September when yo...,Free delivery Pickup by 18 September when you ...,Free return within the legal return period fro...,1 Year Manufacturer Warranty,NaN,NaN
2,Nokia 106 (2018) - 1.8-inch Dual SIM Mobile Ph...,EGP 850.00,3%,EGP 875.00,https://eg.jumia.is/unsafe/fit-in/500x500/filt...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Samsung A24 - 6.5 inches – 6GB/128GB RAM – 4G ...,"EGP 7,399.00",8%,"EGP 7,999.00",https://eg.jumia.is/unsafe/fit-in/500x500/filt...,Technology: GSM / HSPA / LTE 2G bands: GSM 850...,Dimensions: 162.1 x 77.6 x 8.3 mm Weight: 195 ...,"Type: Super AMOLED, 90Hz, 1000 nits (peak) Siz...","OS: Android 13, One UI 5.1 CPU: Octa-core (2x2...",Card slot: microSDXC (dedicated slot) Internal...,"Triple: 50 MP, f/1.8, (wide), PDAF, OIS 5 MP, ...","Single: 13 MP, f/2.2, (wide) Video: 1080p@30fps",Loudspeaker: Yes 3.5mm jack: Yes,"WLAN: Wi-Fi 802.11 a/b/g/n/ac, dual-band, Wi-F...","Sensors: Fingerprint (side-mounted), accelerom...","Li-Po 5000 mAh, non-removable 25W wired","6.56-inch, Super AMOLED, 90Hz, 1000 nits (peak...",شاحن و سماعه خاصة به بالاضافة لتقوية سوكت الشح...,Free delivery Delivery by 18 September when yo...,Free delivery Pickup by 18 September when you ...,Free return within the legal return period fro...,1 Year Manufacturer Warranty,NaN,NaN
4,realme 10 – 6.4 Inch – 128GB/8GB RAM – Dual SI...,"EGP 7,099.00",10%,"EGP 7,899.00",https://eg.jumia.is/unsafe/fit-in/500x500/filt...,Dimensions: 159.9 x 73.3 x 8 mm (6.30 x 2.89 x...,"Type: Super AMOLED, 90Hz, 1000 nits (peak) Siz...","OS: Android 12, Realme UI 3.0 Chipset: Mediate...",Card slot: microSDXC (dedicated slot) Internal...,"Dual: 50 MP, f/1.8, 27mm (wide), 1/2.76"", PDAF...","Single: 16 MP, f/2.5, 27mm (wide) Features: HD...",Loudspeaker: Yes 3.5mm jack: Yes ...,"WLAN: Wi-Fi 802.11 a/b/g/n/ac, dual-band Bluet...","Sensors: Fingerprint (side-mounted), accelerom...","Type : Li-Po 5000 mAh, non-removable Charging:...",NaN,6.4 Inch FHD+ Super AMOLED Display\n128GB Stor...,NaN,NaN,NaN,NaN,NaN,Technology: GSM / HSPA / LTE 2G bands: GSM 850...,NaN
5,realme 10 – 6.4 Inch – 256GB/8GB RAM – Dual SI...,"EGP 7,849.00",13%,"EGP 8,999.00",https://eg.jumia.is/unsafe/fit-in/500x500/filt...,Dimensions: 159.9 x 73.3 x 8 mm (6.30 x 2.89 x...,"Type: Super AMOLED, 90Hz, 1000 nits (peak) Siz...","OS: Android 12, Realme UI 3.0 Chipset: Mediate...",Card slot: microSDXC (dedicated slot) Internal...,"Dual: 50 MP, f/1.8, 27mm (wide), 1/2.76"", PDAF...","Single: 16 MP, f/2.5, 27mm (wide) Features: HD...",Loudspeaker: Yes 3.5mm jack: Yes ...,"WLAN: Wi-Fi 802.11 a/b/g/n/ac, dual-band Bluet...","Sensors: Fingerprint (side-mounted), accelerom...","Type : Li-Po 5000 mAh, non-removable Charging:...",NaN,6.4 Inch FHD+ Super AMOLED Display\n256GB Stor...,NaN,NaN,NaN,Na

In [12]:
df.to_csv('Jumia_mobiles.csv',index=None)

In [121]:
df = pd.read_csv('Jumia_mobiles.csv')

In [122]:
df.columns

Index(['Phone Name', 'Price', 'Discount percentage', 'Price before discount',
       'Image', 'Network', 'Body', 'Display', 'Platform', 'Memory',
       'Main Camera', 'Selfie Camera', 'Sound', 'Communications', 'Features',
       'Battery', 'Specifications', 'Reviews', 'Door delivery',
       'Pickup_Station', 'Return_policy', 'Warranty',
       '                                                                                                   ',
       '                                                                                                '],
      dtype='object')

In [125]:
df.dropna(subset=['Specifications'], inplace=True)

In [126]:
backcamera = []
frontcamera = []
cpu = []
battery = []
os = []
screen = []
storage = []
camera = []
for i in df['Specifications'].dropna():
    spec = i.split('\n')
    screen.append( spec[0])
    storage.append(spec[1])
    camera.append(spec[2])
    cpu.append(spec[3])
    battery.append(spec[4])
    os.append(spec[5])

In [127]:
screensize = []
screentype = []
for i in screen:
    scr = re.split(r'[ -]', i)
    for j in range(len(scr)):
        if 'inch' in scr[j].lower():
            screensize.append(scr[j-1])
        elif 'LCD' in scr[j] or 'AMOLED' in scr[j]:
            screentype.append(scr[j-1] + ' ' +scr[j])


In [128]:
phonestorage = []
ram = []
for i in storage:
    stor = i.split(' ')
    for j in range(len(stor)):
        if 'Storage' in stor[j]:
            phonestorage.append(stor[j-1])
        elif 'RAM' in stor[j]:
            if stor[j-1] == 'GB':
                ram.append(stor[j-2])
            else:
                ram.append(stor[j-1])

In [129]:
front_camera = []
back_camera = []
for camera_info in camera:
    parts = camera_info.split(',')
    for part in parts:
        if 'Front Camera' in part:
            front_camera.append(part.strip())
        elif 'Back Camera' in part:
            back_camera.append(part.strip())


In [131]:
df['front_camera'] = front_camera
df['back_camera'] = back_camera
df['Storage'] = phonestorage
df['RAM'] = ram
df['Screen_size'] = screensize
df['Screen_Type'] = screentype

In [135]:
df

,Phone Name,Price,Discount percentage,Price before discount,Image,Network,Body,Display,Platform,Memory,Main Camera,Selfie Camera,Sound,Communications,Features,Battery,Specifications,Reviews,Door delivery,Pickup_Station,Return_policy,Warranty,,,front_camera,back_camera,Storage,RAM,Screen_size,Screen_Type
1,Samsung Galaxy A14 - 6.6 Inches - 4GB Ram – 12...,"EGP 5,399.00",4%,"EGP 5,600.00",https://eg.jumia.is/unsafe/fit-in/500x500/filt...,Technology: GSM / HSPA / LTE 2G bands: GSM 850...,Dimensions: 167.7 x 78 x 9.1 mm (6.60 x 3.07 x...,"Type: PLS LCD Size: 6.6 inches, 104.9 cm2 (~80...","OS: Android 13, One UI 5 Chipset: Mediatek MT6...",Card slot: microSDXC (dedicated slot) Internal...,"Quad: 50 MP, f/1.8, (wide), PDAF 5 MP, f/2.2, ...","Single: 13 MP, f/2.0, (wide) Video: 1080p@30fps",Loudspeaker: Yes 3.5mm jack: Yes,"WLAN: Wi-Fi 802.11 a/b/g/n/ac, dual-band, Wi-F...","Sensors: Fingerprint (side-mounted), accelerom...","Li-Po 5000 mAh, non-removable Charging: 15W wired","6.6 inches PLS LCD\n128GB Storage, 4 GB RAM\n5...",مناسب لاستخدامى- تليفون مميز في مجال الموبايلات,Free delivery Delivery by 18 September when yo...,Free delivery Pickup by 18 September when you ...,Free return within the legal return period fro...,1 Year Manufacturer Warranty,NaN,NaN,13 MP Front Camera,50 + 5 + 2 MP Back Camera,128GB,4,6.6,PLS LCD
3,Samsung A24 - 6.5 inches – 6GB/128GB RAM – 4G ...,"EGP 7,399.00",8%,"EGP 7,999.00",https://eg.jumia.is/unsafe/fit-in/500x500/filt...,Technology: GSM / HSPA / LTE 2G bands: GSM 850...,Dimensions: 162.1 x 77.6 x 8.3 mm Weight: 195 ...,"Type: Super AMOLED, 90Hz, 1000 nits (peak) Siz...","OS: Android 13, One UI 5.1 CPU: Octa-core (2x2...",Card slot: microSDXC (dedicated slot) Internal...,"Triple: 50 MP, f/1.8, (wide), PDAF, OIS 5 MP, ...","Single: 13 MP, f/2.2, (wide) Video: 1080p@30fps",Loudspeaker: Yes 3.5mm jack: Yes,"WLAN: Wi-Fi 802.11 a/b/g/n/ac, dual-band, Wi-F...","Sensors: Fingerprint (side-mounted), accelerom...","Li-Po 5000 mAh, non-removable 25W wired","6.56-inch, Super AMOLED, 90Hz, 1000 nits (peak...",شاحن و سماعه خاصة به بالاضافة لتقوية سوكت الشح...,Free delivery Delivery by 18 September when yo...,Free delivery Pickup by 18 September when you ...,Free return within the legal return period fro...,1 Year Manufacturer Warranty,NaN,NaN,13 MP Front Camera,50 + 5 + 2 MP Back Camera,128GB,6,6.56,"Super AMOLED,"
4,realme 10 – 6.4 Inch – 128GB/8GB RAM – Dual SI...,"EGP 7,099.00",10%,"EGP 7,899.00",https://eg.jumia.is/unsafe/fit-in/500x500/filt...,Dimensions: 159.9 x 73.3 x 8 mm (6.30 x 2.89 x...,"Type: Super AMOLED, 90Hz, 1000 nits (peak) Siz...","OS: Android 12, Realme UI 3.0 Chipset: Mediate...",Card slot: microSDXC (dedicated slot) Internal...,"Dual: 50 MP, f/1.8, 27mm (wide), 1/2.76"", PDAF...","Single: 16 MP, f/2.5, 27mm (wide) Features: HD...",Loudspeaker: Yes 3.5mm jack: Yes ...,"WLAN: Wi-Fi 802.11 a/b/g/n/ac, dual-band Bluet...","Sensors: Fingerprint (side-mounted), accelerom...","Type : Li-Po 5000 mAh, non-removable Charging:...",NaN,6.4 Inch FHD+ Super AMOLED Display\n128GB Stor...,NaN,NaN,NaN,NaN,NaN,Technology: GSM / HSPA / LTE 2G bands: GSM 850...,NaN,16 MP Front Camera,50 + 2 MP Back Camera,128GB,8,6.4,Super AMOLED
5,realme 10 – 6.4 Inch – 256GB/8GB RAM – Dual SI...,"EGP 7,849.00",13%,"EGP 8,999.00",https://eg.jumia.is/unsafe/fit-in/500x500/filt...,Dimensions: 159.9 x 73.3 x 8 mm (6.30 x 2.89 x...,"Type: Super AMOLED, 90Hz, 1000 nits (peak) Siz...","OS: Android 12, Realme UI 3.0 Chipset: Mediate...",Card slot: microSDXC (dedicated slot) Internal...,"Dual: 50 MP, f/1.8, 27mm (wide), 1/2.76"", PDAF...","Single: 16 MP, f/2.5, 27mm (wide) Features: HD...",Loudspeaker: Yes 3.5mm jack: Yes ...,"WLAN: Wi-Fi 802.11 a/b/g/n/ac, dual-band Bluet...","Sensors: Fingerprint (side-mounted), accelerom...","Type : Li-Po 5000 mAh, non-removable Charging:...",NaN,6.4 Inch FHD+ Super AMOLED Display\n256GB Stor...,NaN,NaN,NaN,NaN,NaN,Technology: GSM / HSPA / LTE 2G bands: GSM 850...,NaN,16 MP Front Camera,50 + 2 MP Bac